In [1]:
#                    Copyright (c) 2020 STMicroelectronics.
#                              All rights reserved,

#    This software component is licensed by ST under BSD-3-Clause license,
#    the "License" You may not use this file except in compliance with the,
#    License. You may obtain a copy of the License at:,
#                        https://opensource.org/licenses/BSD-3-Clause"

# NanoEdge AI Utilities
This script creates normal and abnormal segments for NanoEdge AI studio. These segments can be used to create a library for the given setup.

In this notebook we provide code to take any given datalog from High-Speed Datalogger version 2.0.0 and convert it to Cartesiam complient segments. The converted data is saved as a csv file in the datalog folder.

Another section is provided to combine the data for all the normal and abnormal conditions into single normal and abnormal segment files.

#### Import all the dependencies

In [2]:
import sys, numpy as np
sys.path.append('../../Datalog/Python_SDK/')
from HSD.HSDatalog import HSDatalog
from HSD_utils.converters import HSDatalogConverter

ModuleNotFoundError: No module named 'click'

#### defining all the required variables

In [ ]:
# data information
dataFolder = '../Datasets/HSD_Logged_Data/Fan12CM/'
speeds = [1000, 1250, 1500]
conditions = [ 'normal', 'clogged' ]

# sensor information
sensor_name = 'ISM330DHCX' # other choice is IIS3DWB
sensor_name = 'IIS3DWB' # other choice is ISM330DHCX
sensor_type = 'ACC'
raw = True # if the raw samples are required (True) or sensitivity to be applied to get physical values (False)

dataFolder = dataFolder + sensor_name + '/'

if sensor_name == 'IIS3DWB':
    # framing related variables
    winLen = 4096
    stride = 4096
    offset = 2048
else:
    # framing related variables
    winLen = 1024
    stride = 1024
    offset = 512    

### Converting all the datalogs into cartesiam complient formats, i.e. framing and vectorizing

In [ ]:
for cdn in conditions:
    for spd in speeds:
        acqDir = '{}/{}/{}RPM/'.format( dataFolder, cdn, spd )
        print(acqDir)
        # creating a high-speed datalog object for the given acquisition folder
        hsd = HSDatalog( acquisition_folder = acqDir )
        
        # reading the data as dataframe
        dataFrame = hsd.get_dataframe( sensor_name, sensor_type, sample_start = offset, raw_flag = raw )
        
        # converting to cartesiam complient frames
        HSDatalogConverter.to_cartesiam_format( output_folder = acqDir, sensor_name = sensor_name,
                                               df = dataFrame, signal_length = winLen, signal_increment = stride )

### Combing all the normal and abnormal segments into single files

In [ ]:
# initializing normal and abnormal data variables
normalData = np.empty( ( 0, winLen * 3 ) ) 
abnormalData = np.empty( ( 0, winLen * 3 ) )

# number of samples (if this many available otherwise all the segments used) to use from each acquisition
numSamplesPerAcq = 50

# combining data for all the speeds
for spd in speeds:
    
    # read normal data for a given speed 
    spdNormData = np.genfromtxt( '{}/{}/{}RPM/{}_Cartesiam_segments_0.csv'.format( dataFolder, conditions[0], spd, sensor_name ), 
                                 delimiter = ',')[ :numSamplesPerAcq, : ]
    # concatenating to the data for the other speeds
    normalData = np.concatenate( ( normalData, spdNormData ) )
    
    # read normal data for a given speed
    spdAbnormData = np.genfromtxt( '{}/{}/{}RPM/{}_Cartesiam_segments_0.csv'.format( dataFolder, conditions[0], spd, sensor_name ),
                                   delimiter = ',' )[ :numSamplesPerAcq, : ]

    # concatenating to the data for the other speeds
    abnormalData = np.concatenate( ( abnormalData, spdAbnormData ) )
    
#     print(spdNormData)
# saving the data in the form of segments.csv
np.savetxt( '{}{}_WL{}_segments.csv'.format( dataFolder, conditions[0], winLen ), normalData, delimiter=',' )
np.savetxt( '{}{}_WL{}_segments.csv'.format( dataFolder, conditions[1], winLen ), abnormalData, delimiter=',')